In [2]:
import paramiko
import os

def sbatch(script_path, slurm_path, requirements_path, credentials, data_dir=None):

    # extract script names from paths
    script_name = script_path[script_path.rfind('/'):] if '/' in script_path else script_path
    slurm_name = slurm_path[slurm_path.rfind('/'):] if '/' in slurm_path else slurm_path
    requirements_name = requirements_path[requirements_path.rfind('/'):] if '/' in requirements_path else requirements_path

    # connect to the server
    ssh = paramiko.SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    ssh.connect(
        "login.accre.vanderbilt.edu",
        username="gunnj",
        password="",
    )

    # load Python and create venv for sbatch job
    ssh.exec_command("module load GCCcore/.10.2.0")
    ssh.exec_command("module load Python/3.8.6")
    ssh.exec_command(f"python -m venv {script_name[:script_name.find('.')]}")

    # add requirements to venv
    sftp = ssh.open_sftp()
    remote_path = "/home/gunnj/" + script_name[:script_name.find('.')] + "/"
    print(remote_path)
    sftp.put(requirements_path, remote_path + requirements_name)
    ssh.exec_command("source ${HOME}/" + script_name[:script_name.find('.')] + "/bin/activate")
    ssh.exec_command("pip install -r ${HOME}/" + script_name[:script_name.find('.')] + requirements_name)
    ssh.exec_command("deactivate")

    # add script and slurm files to server 
    sftp.put(script_path, remote_path + script_name)
    sftp.put(slurm_path, remote_path + slurm_name)

    # add contents of data_dir to server
    if data_dir is not None:
        for file in os.listdir(data_dir):

            #TODO: properly handle nested directory structure
            sftp.put(os.path.join(data_dir, file), remote_path + file)

    # run the script
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('sbatch ' + remote_path + slurm_name)

    # check for any error messages
    print(ssh_stderr.read().decode("utf-8"))

    # display the output of the sbatch command and track the job ID
    submission_message = ssh_stdout.read().decode("utf-8")
    job_id = submission_message.split(' ')[-1]
    print(submission_message)

    return job_id

    # when the job is finished, copy the result back to the local machine manually, i guess?